In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk

# from d2l import torch as d2l


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset('lansinuote/ChnSentiCorp', keep_in_memory=True)
        dataset = load_from_disk('./data/ChnSentiCorp')
        
        def f(data):
            return len(data['text']) > 30
        
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        return text

In [3]:
dataset = Dataset('train')

len(dataset) # , dataset[0]

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/validation/cache-cf45964edee402a8.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-c6f7400aef16ddba.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-478819d08c52879a.arrow


3

In [4]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

print(token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def collate_fn(data):
    
    # print(data)
    # print('data长度: ', len(data))
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    # print('编码后的data打印: ', data.keys())
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    # 把第15个词固定替换为mask
    # print('第15个词: %s' % input_ids[:, 15])
    
    # 这里直接使用了编码后的数据作为真实预测值
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    # print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [6]:
# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset['train'],
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True
                                    )


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print('当前loader数据量: ', len(loader))
    print('解码input_ids: ', token.decode(input_ids[0]))
    print('labels: ', token.decode(labels[0]))
    print('参数打印: ', input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    print("")
    if i >= 1:
        break

当前loader数据量:  574
解码input_ids:  [CLS] 品 牌 机 优 势 ， 性 价 比 还 可 以 吧 ， [MASK] 目 前 为 至 运 行 还 算 稳 定 ， 可 进 [SEP]
labels:  到
参数打印:  torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

当前loader数据量:  574
解码input_ids:  [CLS] 1. 大 堂 前 台 服 务 快 捷 ， 效 率 高 [MASK] 2. 客 房 层 和 房 间 空 气 不 好 ， [SEP]
labels:  。
参数打印:  torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])



## 定义下游任务模型

In [7]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        
        # pretrained = pretrained.to(device)

        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        
        out = self.decoder(out.last_hidden_state[:, 15])
        return out


# 加载保存的模型
model_save_path = 'chinese_full_vacant_mission_2023_4_10.pt'
model = Model()
model.load_state_dict(torch.load(model_save_path))

# 模型转移到GPU上
model.to(device)

# list(model.parameters())[0].device  # device(type='cuda', index=0)

Model(
  (decoder): Linear(in_features=768, out_features=21128, bias=True)
)

## 模型测试

In [22]:
#测试
def test_calculate():
    
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset['test'],
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        if i == 15:
            break

        with torch.no_grad():
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)
            
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        
        out = out.argmax(dim=1).cpu()
        labels = labels.cpu()
        
        correct += (out == labels).sum().item()
        total += len(labels)
        
        decode_text = token.decode(labels[0])
        print('序: %s input: %s \t decode: %s' % \
              (i, token.decode(input_ids[0]), decode_text))
        
    print('acc: %.2f' % (correct / total))
    pass

In [23]:
# 预测
test_calculate()

序: 0 input: [CLS] 酒 店 还 可 以 ， 房 内 有 电 脑 ， 数 字 [MASK] 视 。 早 餐 一 般 啦 。 隔 音 效 果 比 [SEP] 	 decode: 电
序: 1 input: [CLS] 装 系 统 太 麻 烦 。 我 主 要 是 office ， 但 [MASK] 键 盘 打 字 很 容 易 跑 空 ， 串 行 。 [SEP] 	 decode: 是
序: 2 input: [CLS] 优 点 : 酒 店 位 置 挺 好, 服 务 生 态 [MASK] 很 好, 早 餐 很 好 缺 点 : 房 间 居 [SEP] 	 decode: 度
序: 3 input: [CLS] 服 务 人 员 素 质 不 高 ， 屡 屡 出 错 ！ [MASK] 明 我 为 同 事 预 留 好 的 房 间 ， 居 [SEP] 	 decode: 明
序: 4 input: [CLS] 这 本 书 有 别 于 以 往 看 过 的 早 教 书 [MASK] ， 结 合 了 说 明 文 的 写 实 ， 散 文 [SEP] 	 decode: 籍
序: 5 input: [CLS] 我 十 一 来 住 了 两 天 ， 一 进 门 就 象 [MASK] 骗 的 感 觉 。 进 旅 馆 有 几 百 米 的 [SEP] 	 decode: 给
序: 6 input: [CLS] 1. 有 急 事 出 去 ， 要 们 童 叫 出 租 [MASK] ， 他 们 就 叫 酒 店 里 的 黑 车 ， 价 [SEP] 	 decode: 车
序: 7 input: [CLS] 没 买 的 就 不 用 买 了 ， 到 新 浪 网 上 [MASK] 书 频 道 看 一 眼 足 够 了 。 毕 大 夫 [SEP] 	 decode: 读
序: 8 input: [CLS] [UNK] 键 在 最 边 上 很 不 习 惯 ， 容 易 和 [MASK] 键 混 淆 ， 音 响 在 键 盘 两 侧 不 舒 [SEP] 	 decode: [ U N K ]
序: 9 input: [CLS] 这 套 书 ， 是 听 别 人 介 绍 的 ， 回 家 [MASK] 3 岁 的 女 儿 一 起 分 享 了 故 事 内 [SEP] 	 decode: 与
序: 10 input: [CL